In [ ]:
import anthropic

client = anthropic.Client(api_key="YOUR API HERE")

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=1,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content[0].text)


Tears of ancient mountains,
Dissolved through time's long sweep,
Minerals and memories
Flow to endless deep.

Where rivers meet the sea,
They bring their earthen load -
Salt of countless ages
In Neptune's vast abode.


In [11]:
import anthropic
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import sys
import os

c:\Users\ALTjasonspeed\.conda\envs\JJAZ\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from prompts.temp_expert_prompt import e_system_prompt, e_user_prompt
from prompts.temp_prompt import system_prompt, user_prompt

client = anthropic.Client(api_key="YOUR API HERE")
def generate_expert(dilema_decription, expert_opionion):
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system=f"{e_system_prompt}",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{e_user_prompt.format(dilema_decription, expert_opionion)}"
                    }
                ]
            }
        ]
    )
    
    return response.content[0].text

def generate_output(dilemma_decription):
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system=f"{system_prompt}",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{user_prompt.format(dilemma_decription)}"
                    }
                ]
            }
        ]
    )
    
    return response.content[0].text

In [ ]:
llm_answers = pd.read_csv("../data/llm_answers.csv")
llm_answers = llm_answers[["Dilemma Description", "Expert Opinion"]]
llm_answers["expert_output"] = llm_answers.apply(lambda row: generate_expert(row['Dilemma Description'], row['Expert Opinion']), axis=1)
llm_answers["dilemma_output"] = llm_answers.apply(lambda row: generate_output(row['Dilemma Description']), axis=1)

temp = llm_answers['expert_output'].str.split('%', expand=True)
temp.columns = ['' if col == 0 else temp[col].str.split(':', expand=True)[0].iloc[0] for col in temp.columns]
for col in temp.columns[1:]:
    try:
        llm_answers["Expert " + col] = temp[col].astype(str).str.split(':', expand=True)[1].str.strip()
    except:
        continue

temp2 = llm_answers['dilemma_output'].str.split('%', expand=True)
temp2.columns = ['' if col == 0 else temp2[col].str.split(':', expand=True)[0].iloc[0] for col in temp2.columns]
for col in temp2.columns[1:]:
    try:
        llm_answers["Dilemma " + col] = temp2[col].astype(str).str.split(':', expand=True)[1].str.strip()
    except:
        continue

llm_answers.to_csv("claude_llm_answers_structured_output.csv")